# Average lidar

In [1]:
# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['/projects/wind_uq/lcheung/amrwind-frontend/',
                  '/ccs/proj/cfd162/lcheung/amrwind-frontend/',
                ]
import sys, os, shutil, io
import numpy as np
for x in amrwindfedirs: sys.path.insert(1, x)

from functools import partial
import matplotlib.pyplot as plt
import pandas as pd

# Make all plots inline 
#%matplotlib inline

import postproengine as ppeng

import imageio
import glob
import cv2

/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/ascldap/users/lcheung/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
# Load ruamel or pyyaml as needed
try:
    import ruamel.yaml
    yaml = ruamel.yaml.YAML(typ='unsafe', pure=True)
    useruamel=True
    loaderkwargs = {'Loader':yaml.RoundTripLoader}
    dumperkwargs = {'Dumper':yaml.RoundTripDumper, 'indent':4, 'default_flow_style':False}
    Loader=yaml.load
except:
    import yaml as yaml
    useruamel=False
    loaderkwargs = {}
    dumperkwargs = {'default_flow_style':False }
    Loader=yaml.safe_load
    
def stringReplaceDict(s, dreplace):
    outstr = str(s)
    for k, g in dreplace.items():
        outstr=outstr.replace(k, str(g))
    return outstr
    

In [3]:
replacedict={'RUNDIRA':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/',
             'RUNDIR_A1':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1/',
             'RUNDIR_A2':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2/',
             'RUNDIR_A3':'/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3/',
             #'NCPREFIX':'hubheight',
             'NCPREFIX':'KP_z090hh',
            }

In [4]:
yamlstring="""
globalattributes:
  verbose: False
  executeorder:
  - Reavgworkflow_C1a

filelist: &nclist [RUNDIRA/post_processing/lidar_14400.nc, RUNDIR_A1/post_processing/lidar_37800.nc, RUNDIR_A2/post_processing/lidar_54000.nc, RUNDIR_A3/post_processing/lidar_72900.nc]
varlist: &vlist ['velocityx', 'velocityy', 'velocityz',]

Reavgworkflow_C1a:
  linesampler:
  - name: lidar_siteC1a
    ncfile: *nclist
    group: lidar_siteC1a
    varnames: *vlist
    reynoldsstress: 
      tavg: [8400, 9000]
      savefile: AVG_lidar/siteC1a_RESTRESS_9000.csv
  - name: lidar_siteC1a
    ncfile: *nclist
    group: lidar_siteC1a
    varnames: *vlist
    reynoldsstress: 
      tavg: [10200, 10800]
      savefile: AVG_lidar/siteC1a_RESTRESS_1800.csv
  - name: lidar_siteC1a
    ncfile: *nclist
    group: lidar_siteC1a
    varnames: *vlist
    reynoldsstress: 
      tavg: [12000, 12600]
      savefile: AVG_lidar/siteC1a_RESTRESS_12600.csv
  - name: lidar_siteC1a
    ncfile: *nclist
    group: lidar_siteC1a
    varnames: *vlist
    reynoldsstress: 
      tavg: [13800, 14400]
      savefile: AVG_lidar/siteC1a_RESTRESS_14400.csv

Reavgworkflow_A1H:
  linesampler:
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: *vlist
    reynoldsstress: 
      tavg: [8400, 9000]
      savefile: AVG_lidar/siteH_RESTRESS_9000.csv
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: *vlist
    reynoldsstress: 
      tavg: [10200, 10800]
      savefile: AVG_lidar/siteH_RESTRESS_10800.csv
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: *vlist
    reynoldsstress: 
      tavg: [12000, 12600]
      savefile: AVG_lidar/siteH_RESTRESS_12600.csv
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: *vlist
    reynoldsstress: 
      tavg: [13800, 14400]
      savefile: AVG_lidar/siteH_RESTRESS_14400.csv

  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: *vlist
    reynoldsstress: 
      tavg: [8400, 9000]
      savefile: AVG_lidar/siteA1_RESTRESS_9000.csv
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: *vlist
    reynoldsstress: 
      tavg: [10200, 10800]
      savefile: AVG_lidar/siteA1_RESTRESS_10800.csv
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: *vlist
    reynoldsstress: 
      tavg: [12000, 12600]
      savefile: AVG_lidar/siteA1_RESTRESS_12600.csv
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: *vlist
    reynoldsstress: 
      tavg: [13800, 14400]
      savefile: AVG_lidar/siteA1_RESTRESS_14400.csv

timeavgworkflow:
  linesampler:
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [8400, 9000]
      savefile: AVG_lidar/siteH_AVG_9000.csv
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [10200, 10800]
      savefile: AVG_lidar/siteH_AVG_10800.csv
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [12000, 12600]
      savefile: AVG_lidar/siteH_AVG_12600.csv
  - name: lidar_siteH
    ncfile: *nclist
    group: lidar_siteH
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [13800, 14400]
      savefile: AVG_lidar/siteH_AVG_14400.csv
  
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [8400, 9000]
      savefile: AVG_lidar/siteA1_AVG_9000.csv
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [10200, 10800]
      savefile: AVG_lidar/siteA1_AVG_10800.csv
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [12000, 12600]
      savefile: AVG_lidar/siteA1_AVG_12600.csv
  - name: lidar_siteA1
    ncfile: *nclist
    group: lidar_siteA1
    varnames: ['velocityx', 'velocityy', 'velocityz', 'temperature']
    average: 
      tavg: [13800, 14400]
      savefile: AVG_lidar/siteA1_AVG_14400.csv

"""
f = io.StringIO(stringReplaceDict(yamlstring, replacedict))
yamldict = Loader(f, **loaderkwargs)

In [ ]:
# Run the driver
ppeng.driver(yamldict, verbose=True)

Initialized linesampler
Running linesampler
Initialized reynoldsstress inside linesampler
Executing reynoldsstress
first ncfilelist  ['/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA//post_processing/lidar_14400.nc', '/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1//post_processing/lidar_37800.nc', '/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc', '/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc']
Calculating averages
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA//post_processing/lidar_14400.nc 6000


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[============================================================] 100.0% 
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1//post_processing/lidar_37800.nc 0

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc 0

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc 0

Ncount = 6000

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA//post_processing/lidar_14400.nc 6000
Calculating reynolds-stress
[============================================================] 100.0% 
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1//post_processing/lidar_37800.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[============================================================] 100.0% 
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc 0

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc 0

Ncount = 6000

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA//post_processing/lidar_14400.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1//post_processing/lidar_37800.nc 6000
Calculating reynolds-stress
[============================================================] 100.0% 
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc 0
Calculating reynolds-stress

Ncount = 6000

Initialized reynoldsstress inside linesampler
Executing

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[============================================================] 100.0% 
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc 0

Ncount = 6000

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA//post_processing/lidar_14400.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1//post_processing/lidar_37800.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc 6000
Calculating reynolds-stress
[============================================================] 100.0% 
/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc 0
Calculating reynolds-stress

Ncount = 6000

Initialized reynoldsstress inside linesampler
Executing reynoldsstress
first ncfilelist  ['/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA/

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[============================================================] 100.0% 
Ncount = 5985

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA//post_processing/lidar_14400.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA1//post_processing/lidar_37800.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA2//post_processing/lidar_54000.nc 0
Calculating reynolds-stress

/tscratch/lcheung/AWAKEN/Benchmark1/Phase3/FarmRuns/BM3_FarmRunProd1_runA3//post_processing/lidar_72900.nc 5985
Calculating reynolds-stress
